#OA3802 Lab 5: GDELT (Geohash)
###Travis Farwell
##Task 0: Access GDELT Data
The first task is to fetch entries from the GDELT (Global Database of Events, Language, and Tone) database for a given date and 30 days prior. First some modules are import into the work space. 

In [1]:
import re
import pytz
import gdelt
import math
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import geoplot as gplt
import pygeohash as pgh
from tzwhere import tzwhere 
from haversine import haversine
import matplotlib.pyplot as plt
from pytrie import SortedStringTrie as Trie
import warnings
warnings.filterwarnings('ignore')

tz1 = tzwhere.tzwhere(forceTZ=True)

Now a function is created that takes three parameters as input; a year, a month and a day all in numberic form. Inside the function a gdelt object is created which will be used to search the database. The inputted parameters are converted into a 2 datetime objects. A start date and an end date using the date() and timedelta() functions are created which are passed to the Search() function from the gdelt library. The results are stored as a pandas dataframe.

In [2]:
def pull_30(year, month, day):
    gd = gdelt.gdelt()
    start_date = datetime.date(year,month,day) - datetime.timedelta(30)
    end_date = datetime.date(year,month,day)
    return gd.Search([str(start_date),str(end_date)],output='gpd',normcols=True,coverage=False)
    
results = pull_30(2017,10,31)

##Task 1: Great-Circle Distance Subsetting

Given latitude and longitude coordinates, a distance in kilometers and a list of numeric CAMEO event codes, pull the corresponding events which are within the great-circle of these coordinates. The function below loops through each event calculating the distance of the event and the given coordinates. The haversine library is used as a helper function to calculate the distance. If the distance is within the desired great-circle and the eventcode is in the list of desired eventcodes, the event is appended to an output list.  A dataframe is created from the list and sorted by distance. The top 10 closest events are returned. 

In [3]:
def great_circle(x = (15.05,1.82), y = 300, z = [13,14,18,19,20]):
    output = []
    for i in range(len(results)):
        try:
            dist = round(haversine(x,(results['actiongeolat'][i],results['actiongeolong'][i])),2)
            if dist <= y:
                if int(results['eventcode'][i]) in z:
                    output.append([results['eventcode'][i],(round(results['actiongeolat'][i],2),\
                                                            round(results['actiongeolong'][i],2)),dist])
        except:
            continue
    df = pd.DataFrame(output)
    df.columns = ['EventCode', 'Lat-Long','Distance']
    df.sort_values('Distance')
    return df[0:10]
print(great_circle())

  EventCode       Lat-Long  Distance
0       013  (15.05, 1.83)      1.29
1       020  (13.52, 2.12)    173.47
2       020  (13.52, 2.12)    173.47


##Task 2: Geohash Subsetting

The end goal of task 2 is the same as task 1 but this time using geohashing to find events rather than great-circle distance. The pygeohash library is used to create the geohashes from the latitude and longitude coordinates. A hash will be found beforehand and added to each event. The encode function is vectorized so the geohashes can be created on the dataframe in parallel using a lambda expression inside the pandas assign() function. From the marisa_trie library, a trie data structure will be created to serve to query matching hashes faster. A trie is perfect for geohashing as geohashing works by finding matching prefixes. The more matching prefixes two geohashes have the closer the two points are on the map. 

In [4]:
#vectorize encode function
vect = np.vectorize(pgh.encode)
results = results.assign(geohash = lambda x: vect(x.actiongeolat,x.actiongeolong) )
results = results.sort_values('geohash')

import marisa_trie
trie = marisa_trie.Trie(list(results['geohash']))

Geohashing isn't as precise as great-circle calculation. In the geohash() function below, a list of errors is created which represents the precision as more geohash prefixes are matched. For example, if 1 prefix is matched the two points are at least within 2500 km of eachother. If 2 prefixes are matched at least 630 km of eachother and so on. In the function below the number of desired prefix mathes is calculated by looping through the errors list and stopping when the desired distance (default is y = 300) is less than the errors index. This number will be used to find all events that at least match this number of prefixes. In the example we want to match at least 2 indexes, i.e. everything less than 630 km precision. A geohash is created of the desired center. The desired prefix match is sent to the keys() function to fetch all keys from the trie that match the prefix.  From these, the great-circle distance is computed to find a precise solution since some of the hash distances will be more than the desired radius.

In [5]:
def geohash(x = (15.05,1.82), y = 300, z = [13,14,18,19,20]):
    #create list of geohashes
    output = []
    #geohash error precision
    errors = [2500,630,78,20,2.4,.61,.076,.019]
    i = 0
    while y < errors[i]:
        i = i + 1
    center = pgh.encode(x[0],x[1])
    #fetch all matching prefixes of length i or better
    keys = trie.keys(center[0:(i)])
    for i in range(len(results)):
        try:
            if str(results['geohash'][i]) in keys:
                dist = round(haversine(x,(results['actiongeolat'][i],results['actiongeolong'][i])),2)
                if dist < y and int(results['eventcode'][i]) in z:
                    output.append([results['geohash'][i],\
                                   results['eventcode'][i],\
                                   (round(results['actiongeolat'][i],2),\
                                    round(results['actiongeolong'][i],2)),\
                                   dist, results['sourceurl'][i]])
        except:
            continue
    df = pd.DataFrame(output)
    df.columns = ['Geohash','EventCode', 'Lat-Long','Distance','Url']
    df.sort_values('Distance')
    return df[0:10]        
print(geohash())              

        Geohash EventCode       Lat-Long  Distance  \
0  s49m9sjvkj3n       013  (15.05, 1.83)      1.29   
1  s43sbhtk3zx8       020  (13.52, 2.12)    173.47   
2  s43sbhtk3zx8       020  (13.52, 2.12)    173.47   

                                                 Url  
0  http://www.tbo.com/ap/national/us-general-lays...  
1  http://dailycaller.com/2017/10/23/dunford-reve...  
2  http://dailycaller.com/2017/10/23/dunford-reve...  


##Task 3: Benchmarking and Implementation

The performance of each technique are timed. The geohash function takes about half the time as the great-circle implementation. The geohash searches at O(logn) while the great-circle has to compute the distance between every entry and the desired center making it's computation complexity O(n). One drawback of the geohash method is that the geohashes need to be computed first and stored before they can be searched efficiently. This takes up more memory than the great-circle way. Another possible issue with the geohash computation is that precision is sacrificed for speed. By adding a haversine calculation in the geohash function after the search is completed, more precise entries can be returned. This implementation was used in the geohash function in Task 2. 

In [6]:
print("Timing of Great_Circle Implementation")
%time great_circle()
print("\nTimeing of Geohash Implementation")
%time geohash()

Timing of Great_Circle Implementation
CPU times: user 1.66 s, sys: 0 ns, total: 1.66 s
Wall time: 1.66 s

Timeing of Geohash Implementation
CPU times: user 796 ms, sys: 0 ns, total: 796 ms
Wall time: 798 ms


,Geohash,EventCode,Lat-Long,Distance,Url
0,s49m9sjvkj3n,013,"(15.05, 1.83)",1.29,http://www.tbo.com/ap/national/us-general-lays...
1,s43sbhtk3zx8,020,"(13.52, 2.12)",173.47,http://dailycaller.com/2017/10/23/dunford-reve...
2,s43sbhtk3zx8,020,"(13.52, 2.12)",173.47,http://dailycaller.com/2017/10/23/dunford-reve...


##Task 4: Model Outputs

Using the information recieved from a search we are able to use the coordinates to plot to a map using the folium library. A function is used that uses the same parameters as before and passed to the geohash() function. A map is created using the Map() method. Now we can loop through the events and place a marker at the corresponding lat/long points. A popup is added to the Marker() method which adds the urls to each marker so that the viewer can find the source where the event came from. Some randomness is added to the coordinates so that the markers are not on top of eachother.

In [7]:
import folium
import random

def get_map(x = (33,43), y = 250, z = [13,14,18,19,20]):
    df = geohash(x,y,z)
    map_1 = folium.Map(location=[x[0],x[1]],zoom_start =8)
    for i in range(len(df)):
        lat = df['Lat-Long'][i][0]
        long = df['Lat-Long'][i][1]
        #Add some randomness
        lat_ran = (random.random()*2 -1)/20
        long_ran = (random.random()*2 - 1)/20
        folium.Marker(location = [lat + lat_ran,long+long_ran], popup = df['Url'][i]).add_to(map_1)
    return map_1

m = get_map()
m


The below code prints out a table in html format with information for each event that may be of importance to the end user. 

In [9]:
from IPython.core.display import display, HTML 
import tabulate
df = geohash(x = (33,43), y = 250, z = [13,14,18,19,20])
display(HTML(tabulate.tabulate(df , tablefmt='html', headers = df.columns.values,showindex=False,floatfmt=".2f")))

Geohash,EventCode,Lat-Long,Distance,Url
svztdjnekv7p,013,"(33.340000000000003, 44.390000000000001)",135.09,http://nypost.com/2017/10/03/team-trump-can-still-prevent-the-next-mideast-explosion/
svz79pryfmcg,020,"(33.0, 44.0)",93.26,https://www.opednews.com/articles/Dylan-s-Alembic-Against-t-by-Chris-Floyd-Bob-Dylan_Mortality_Songs_Women-171005-212.html
svztdjnekv7p,014,"(33.340000000000003, 44.390000000000001)",135.09,https://www.voanews.com/a/turkey-president-erdogan-vows-tough-sanctions-to-thwart-iraqi-kurdish-independence/4058294.html
svz79pryfmcg,020,"(33.0, 44.0)",93.26,https://www.opednews.com/articles/Dylan-s-Alembic-Against-t-by-Chris-Floyd-Bob-Dylan_Mortality_Songs_Women-171005-212.html
svz79pryfmcg,014,"(33.0, 44.0)",93.26,http://gazette.com/turkey-iraq-kurds/article/feed/499238
svz79pryfmcg,014,"(33.0, 44.0)",93.26,http://gazette.com/turkey-iraq-kurds/article/feed/499238
svz79pryfmcg,020,"(33.0, 44.0)",93.26,http://gazette.com/turkey-iraq-kurds/article/feed/499238
svz79pryfmcg,013,"(33.0, 44.0)",93.26,http://beta.latimes.com/la-fg-missing-soldier-found-20171006-story.html
svztdjnekv7p,020,"(33.340000000000003, 44.390000000000001)",135.09,http://iphonefresh.com/2017/10/10/founder-of-security-firm-blackwater-considering-senate-run.html
svztdjnekv7p,020,"(33.340000000000003, 44.390000000000001)",135.09,http://iphonefresh.com/2017/10/10/founder-of-security-firm-blackwater-considering-senate-run.html
